<a href="https://colab.research.google.com/github/fukushun1994/RNN-Anime-model/blob/master/RNN_Anime_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 環境構築


## TPUの設定

ランタイム > ランタイムのタイプを変更 で ハードウェアアクセラレータをTPUに設定



Google Colab でTPUが使えるかチェック

In [0]:
import os
import pprint
import tensorflow as tf

if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()
    
  print('TPU devices:')
  pprint.pprint(devices)

TPU address is grpc://10.61.108.146:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 6451243092619256862),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 12687380745649448051),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 3001828563917283008),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 10556619114126885709),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 288463987303251174),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 6453442571066480757),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 17199960894891622339),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 1712409129858502805),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 12326

# データの準備


## Google Drive のマウント

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


## データの取得

In [13]:
import zipfile
import numpy as np
from PIL import Image

DATA_DIR = '/content/gdrive/My Drive/YuyushikiData/'
ZIP_FILE_NAMES = ['ep1.zip']

dataset = []
for zip_file_name in ZIP_FILE_NAMES:
  data = []
  with zipfile.ZipFile(DATA_DIR+zip_file_name, 'r') as zp:
    image_count = len(zp.namelist())
    print('open {} ({} images)'.format(zip_file_name, image_count))
    for i, image_name in enumerate(sorted(zp.namelist())):
      print('\r{}/{}'.format(i, image_count), end='')
      with zp.open(image_name, 'r') as fp:
        img_vec = np.array(Image.open(fp))
        data.append(img_vec[:,:,:3])
  dataset.append(data)
print('')

print(len(dataset[0]))
print(dataset[0][0].shape)
print(dataset[0][0])

open ep1.zip (26235 images)
26234/26235
26235
(72, 128, 3)
[[[1 0 1]
  [1 0 1]
  [1 0 1]
  ...
  [1 0 1]
  [1 0 1]
  [1 0 1]]

 [[1 0 1]
  [1 0 1]
  [1 0 1]
  ...
  [1 0 1]
  [1 0 1]
  [1 0 1]]

 [[1 0 1]
  [1 0 1]
  [1 0 1]
  ...
  [1 0 1]
  [1 0 1]
  [1 0 1]]

 ...

 [[1 0 1]
  [1 0 1]
  [1 0 1]
  ...
  [1 0 1]
  [1 0 1]
  [1 0 1]]

 [[1 0 1]
  [1 0 1]
  [1 0 1]
  ...
  [1 0 1]
  [1 0 1]
  [1 0 1]]

 [[1 0 1]
  [1 0 1]
  [1 0 1]
  ...
  [1 0 1]
  [1 0 1]
  [1 0 1]]]


# モデルの構築
